<a href="https://colab.research.google.com/github/udianand/DataEngineering/blob/main/Farfetch_Submission_Udit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your pipeline needs to:

Download and parse the YAML config file from the static URL to get the 
filenames that need to be processed
Download and uncompress each gzip file referenced in the yaml
Find the top 5 countries with the biggest number of rows in the source data
Dump the data for those 5 countries to a single file in the disk

In [ ]:
import requests
import zipfile
import io
import gzip
import urllib


In [ ]:
yaml_file = 'https://raw.githubusercontent.com/pmjacinto/hltdata/main/data.yaml'
res = requests.get(yaml_file)

In [ ]:

res_string = res.content.decode("utf-8")


In [ ]:
list_result = res_string.splitlines()

In [ ]:
file_identifier = '- name'
list_of_files = []
for cont in list_result:
  if file_identifier in cont:
    file_names = cont.split(':')
    list_of_files.append(file_names[1])

In [ ]:
print(list_of_files)

[' covid-xaa.json.gz', ' covid-xab.json.gz', ' covid-xac.json.gz', ' covid-xad.json.gz', ' covid-xae.json.gz', ' covid-xaf.json.gz', ' covid-xag.json.gz', ' covid-xah.json.gz', ' covid-xai.json.gz', ' covid-xaj.json.gz', ' covid-xak.json.gz', ' covid-xal.json.gz', ' covid-xam.json.gz', ' covid-xan.json.gz', ' covid-xao.json.gz', ' covid-xap.json.gz']


In [ ]:
!mkdir covid_data

In [ ]:
base_url = 'https://raw.githubusercontent.com/pmjacinto/hltdata/main/'
base_path = '/content/covid_data/'

for files in list_of_files:
  file_name = files.strip()
  zip_file_url = base_url + file_name
  with urllib.request.urlopen(zip_file_url) as response:
    with gzip.GzipFile(fileobj=response) as uncompressed:
      file_content = uncompressed.read()

    #write to file in binary mode 'wb
  file_path = base_path + file_name[:-3]
  with open(file_path, 'wb') as f:
    f.write(file_content)
             

In [ ]:
!head -10 covid_data/covid-xaa.json

{"date":"2020-02-07","location_key":"US_WY_56007","country_code":"US","country_name":"United States of America","subregion1_code":"WY","subregion1_name":"Wyoming","subregion2_code":"56007","subregion2_name":"Carbon County","iso_3166_1_alpha_2":"US","iso_3166_1_alpha_3":"USA","aggregation_level":"2","new_confirmed":"0","new_deceased":"0","new_tested":"0","cumulative_confirmed":"0","cumulative_deceased":"0","cumulative_tested":"0","wikidata_id":"Q156452","datacommons_id":"geoId/56007","openstreetmap_id":"288626","latitude":41.69,"longitude":-106.93,"location_geometry":"POINT(-106.93 41.69)","average_temperature_celsius":-3.506944,"minimum_temperature_celsius":-10.333333,"maximum_temperature_celsius":0.770833,"rainfall_mm":2.709333,"snowfall_mm":149.86}
{"date":"2020-02-08","location_key":"US_WY_56007","country_code":"US","country_name":"United States of America","subregion1_code":"WY","subregion1_name":"Wyoming","subregion2_code":"56007","subregion2_name":"Carbon County","iso_3166_1_alph

In [ ]:
import os
import json
import collections
country_count = collections.defaultdict(int)
covid_file = os.listdir('/content/covid_data/')
for file_name in covid_file:
  try:
    file_path  = '/content/covid_data/'+file_name
    f = open(file_path, encoding = 'utf-8')
    lines = f.readlines()
    for line in lines:
      data = json.loads(line)
      country_count[data['country_code']] += 1
  finally:
    f.close()

In [ ]:
top_five_countries_count = sorted(country_count.items(), key = lambda x:-x[1])[:5]
top_five_countries = []
for country_code,num_rows in top_five_countries_count:
  top_five_countries.append(country_code)

print(top_five_countries)

['BR', 'US', 'MX', 'PE', 'IL']


In [ ]:
import os
import json
import collections

country_data = collections.defaultdict(list)
covid_file = os.listdir('/content/covid_data/')

base_path = '/content/'
output_file_name = 'output'
output_file_path =  base_path + output_file_name

for file_name in covid_file:
  try:
    file_path  = '/content/covid_data/'+file_name
    f = open(file_path, encoding = 'utf-8')
    lines = f.readlines()
    for line in lines:
      data = json.loads(line)
      if data['country_code'] in top_five_countries:
        with open(output_file_path, 'a') as f1:
          f1.write(json.dumps(data))
  finally:
    f1.close()
    f.close()


In [ ]:
!ls -lh output

-rw-r--r-- 1 root root 3.5G Jul  6 19:48 output


In [ ]:
!wc -L output

3679515816 output


In [ ]:
!more -10 output

{"date": "2020-01-01", "location_key": "US", "country_code": "US", "country_name
": "United States of America", "iso_3166_1_alpha_2": "US", "iso_3166_1_alpha_3":
 "USA", "aggregation_level": "0", "new_confirmed": "0", "new_deceased": "0", "cu
mulative_confirmed": "0", "cumulative_deceased": "0", "wikidata_id": "Q30", "dat
acommons_id": "country/USA", "openstreetmap_id": "148838", "latitude": 39.828175
, "longitude": -98.5795, "location_geometry": "POINT(-98.5795 39.828175)", "aver
age_temperature_celsius": -0.975309, "minimum_temperature_celsius": -5.91358, "m
aximum_temperature_celsius": 6.314815, "rainfall_mm": 0, "snowfall_mm": 64.77}{"
date": "2020-01-02", "location_key": "US", "country_code": "US", "country_name":
 "United States of America", "iso_3166_1_alpha_2": "US", "iso_3166_1_alpha_3": "
--More--(0%)